In [1]:
import pandas as pd
import numpy as np
import faiss
from sklearn.metrics.pairwise import cosine_similarity

主要目标是：缩减先进制造和传统制造的行业，因为在做数据描述性统计分析的时候发现，有些行业的部分tech_net方法的效果不好（主要表现在传统制造的标准差比较小，先进制造的有些tech_net为0），所以最好把这部分0值删去。
所以先做下面几步操作 
1、把invest_patent_先进制造_202410_4和传统制造部分里面，tech_net和ipc方法计算closeness等于0的部分筛选之后看一下对应的行业分布（主要集中在哪几个行业）
2、之前我们在计算explorative/exploitative的时候是不是又设置过一些阈值，这部分阈值能不能设置一个算法，自动优化一下让他们的区分度变大？

In [2]:
invest_co_ct=pd.read_excel('data/invest_patent_传统制造_202410_3.xlsx',index_col=0)
invest_co_xj=pd.read_excel('data/invest_patent_先进制造_202410_3.xlsx',index_col=0)
invest_co_xj.columns

Index(['行业', '子行业', '企业全称', '交易时间', '交易轮次', 'all', 'peer_tech_net_closeness_1',
       'peer_tech_net_closeness_2', 'peer_tech_net_closeness_3', 'explorative',
       'exploitative', 'jianjin_patent_num', 'peer_ipc_closeness_1',
       'peer_ipc_closeness_2', 'peer_ipc_closeness_3', '首次上市日期', '是否上市',
       'tech_net_Explorative1_0.7', 'tech_net_Exploitative1_0.7',
       'tech_net_Explorative1_0.75', 'tech_net_Exploitative1_0.75',
       'tech_net_Explorative1_0.8', 'tech_net_Exploitative1_0.8',
       'tech_net_Explorative1_0.85', 'tech_net_Exploitative1_0.85',
       'tech_net_Explorative2_0.7', 'tech_net_Exploitative2_0.7',
       'tech_net_Explorative2_0.75', 'tech_net_Exploitative2_0.75',
       'tech_net_Explorative2_0.8', 'tech_net_Exploitative2_0.8',
       'tech_net_Explorative2_0.85', 'tech_net_Exploitative2_0.85',
       'tech_net_Explorative3_0.7', 'tech_net_Exploitative3_0.7',
       'tech_net_Explorative3_0.75', 'tech_net_Exploitative3_0.75',
       'tech_net_Explorative

In [3]:
(invest_co_ct['peer_tech_net_closeness_1']==0).sum(),(invest_co_ct['peer_ipc_closeness_1']==0).sum()

(np.int64(0), np.int64(109))

In [4]:
(invest_co_ct['peer_tech_net_closeness_2']==0).sum(),(invest_co_ct['peer_tech_net_closeness_3']==0).sum()

(np.int64(0), np.int64(0))

In [7]:
invest_co_ct=pd.read_excel('data/invest_patent_传统制造_202410_4.xlsx',index_col=0)
invest_co_xj=pd.read_excel('data/invest_patent_先进制造_202410_4.xlsx',index_col=0)
(invest_co_ct['peer_tech_net_closeness_1']==0).sum(),(invest_co_ct['peer_tech_net_closeness_2']==0).sum(),(invest_co_ct['peer_tech_net_closeness_3']==0).sum()

(np.int64(0), np.int64(0), np.int64(0))

In [8]:
(invest_co_xj['peer_tech_net_closeness_1']==0).sum(),(invest_co_xj['peer_tech_net_closeness_2']==0).sum(),(invest_co_xj['peer_tech_net_closeness_3']==0).sum()

(np.int64(14071), np.int64(12364), np.int64(11846))

In [6]:
(invest_co_ct['peer_ipc_closeness_1']==0).sum(),(invest_co_ct['peer_ipc_closeness_1']==1).sum()

(np.int64(109), np.int64(7249))

In [9]:
invest_co_ct.loc[invest_co_ct['peer_ipc_closeness_1']==0,['子行业','peer_tech_net_closeness_1']]

,子行业,peer_tech_net_closeness_1
136,传统制造其他,0.724735
143,机械设备,0.691970
378,传统制造其他,0.652305
424,传统制造其他,0.644640
1392,机械设备,0.678238
...,...,...
18377,轻工制造,0.638435
18662,纺织业,0.615392
18853,机械设备,0.652083
19238,轻工制造,0.634813


In [10]:
doc_lda_w2v_xj=pd.read_parquet(f'data\\documents_vec_invest_mfg_xj.parquet')
doc_lda_w2v_xj.shape

(1410528, 300)

In [14]:
(doc_lda_w2v_xj.sum(axis=1)==0).sum()

np.int64(0)

In [16]:
del invest_co_xj['peer_tech_net_closeness_1']
del invest_co_xj['peer_tech_net_closeness_2']
del invest_co_xj['peer_tech_net_closeness_3']

In [25]:
del invest_co_xj['peer_ipc_closeness_1']
del invest_co_xj['peer_ipc_closeness_2']
del invest_co_xj['peer_ipc_closeness_3']

In [2]:
invest_co=pd.read_excel('data\\invest_mfg_co.xlsx',index_col=0)

In [3]:
patent_co=pd.read_csv('data\\patent_co_invest_mfg_2043_15k_202410.csv',index_col=0)
patent_co.shape

(2043777, 35)

In [19]:
patent_co.columns

Index(['标题 (中文)', '摘要 (中文)', '申请人', '申请日', '申请号', '公开（公告）号', '公开（公告）日', '专利类型',
       '专利有效性', '公开国别', 'IPC主分类', 'IPC', '申请人类型', '发明人', '发明人数量', '简单同族',
       '扩展同族', '简单同族个数', '引证专利', '引证科技文献', '引证次数', '被引证次数', '引证申请人', '被引证申请人',
       '申请人数量', '许可次数', '转让次数', '权利要求数量', '当前法律状态', '当前权利人', '工商统一社会信用代码',
       'table_name', '申请年', '企业全称', '权利人'],
      dtype='object')

In [4]:
patent_co['公开（公告）日']=pd.to_datetime(patent_co['公开（公告）日'])
patent_co['申请日']=pd.to_datetime(patent_co['申请日'])
patent_co['Co'] = np.where(pd.notna(patent_co.企业全称),patent_co.企业全称,patent_co.权利人)
patent_co['传统制造']=patent_co['Co'].isin(invest_co.loc[invest_co.行业=='传统制造','企业全称']).astype(int)
patent_co['先进制造']=patent_co['Co'].isin(invest_co.loc[invest_co.行业=='先进制造','企业全称']).astype(int)
patent_co['传统制造'].sum(),patent_co['先进制造'].sum()

(np.int64(1037773), np.int64(1401478))

In [7]:
patent_co=patent_co[['公开（公告）日','公开（公告）号','申请年','申请日','申请号','专利类型','企业全称','Co','传统制造','先进制造']].copy()

In [24]:
invest_co_xj.交易时间

0       2017-03-19
1       2017-03-18
2       2017-03-17
3       2017-03-17
4       2017-03-16
           ...    
27302   2024-01-16
27303   2023-05-12
27304   2022-10-21
27305   2024-03-01
27306   2024-03-01
Name: 交易时间, Length: 27307, dtype: datetime64[ns]

In [7]:
patent_co['技术大类'] =patent_co.IPC主分类.str.strip().str[:3]
patent_co['引证专利_A'] = patent_co['引证专利'].fillna('').str.split(';').apply(lambda x: [i.strip() for i in x if i.strip()])

In [ ]:
%%time
knowledge_year=5
# invest_co_ct=invest_co[invest_co.行业=='传统制造'].copy()#.sort_values(by='交易时间')

for year in sorted(invest_co_xj.交易时间.dt.year.unique()):
    max_date=pd.to_datetime(year, format='%Y')+ pd.offsets.YearEnd()
    filter_invest = invest_co_xj[invest_co_xj.交易时间.dt.year==year]
    filter_patent = patent_co[(patent_co.先进制造==1)&(patent_co.申请年 <= year)]# <= data
    
    if filter_patent.empty:
        continue

    for n in [1,2,3]:
        mask_base=filter_patent.申请日 <  max_date-pd.Timedelta(days=n*365) # pd.Timestamp(year) - pd.DateOffset(years=n)date- pd.DateOffset(years=3)
        
        base_ipc_category =  set(filter_patent.loc[mask_base, '技术大类'].dropna().unique())# filter_patent[mask_base].groupby('Co')['技术大类'].dropna().apply(set).to_dict(), base_ipc_category.get(group.name, set())
        tech_patents = filter_patent[~mask_base].copy()
        tech_patents['in_baseipc']=tech_patents['技术大类'].apply(lambda ipc: ipc in base_ipc_category)
        ipc_similarity=tech_patents.groupby('Co')['in_baseipc'].mean().fillna(0)
        
        invest_co_xj.loc[filter_invest.index,f'peer_ipc_closeness_{n}'] =  filter_invest['企业全称'].map(ipc_similarity)#ipc_similarity[n].get(x, 0)
        
        group_ids_t = filter_patent[mask_base].groupby('Co')['公开（公告）号'].apply(list)
        base_tech_net = group_ids_t.apply(lambda x: doc_lda_w2v_xj.loc[doc_lda_w2v_xj.index.isin(x)].mean(axis=0))
     
        group_ids_t = filter_patent[~mask_base].groupby('Co')['公开（公告）号'].apply(list)
        tech_net_category = group_ids_t.apply(lambda x: doc_lda_w2v_xj.loc[doc_lda_w2v_xj.index.isin(x)].mean(axis=0))
        
        if not tech_net_category.empty and not base_tech_net.empty:
            tech_net_category = tech_net_category[tech_net_category.sum(axis=1) != 0]
            base_tech_net = base_tech_net[base_tech_net.sum(axis=1) != 0]
            cosine_sim_values = cosine_similarity(tech_net_category.values, base_tech_net.values)
            cosine_sim = pd.Series(cosine_sim_values.mean(axis=1), index=tech_net_category.index)
            invest_co_xj.loc[filter_invest.index,  f'peer_tech_net_closeness_{n}']=filter_invest['企业全称'].map(lambda x: cosine_sim.get(x, pd.NA))
     
    # for index,row in filter_invest.iterrows():
    #     company_name = row['企业全称']
    #     date = row['交易时间']

    #     # for n in [1,2,3]:
    #     #     invest_co_ct.at[index,f'peer_ipc_closeness_{n}'] = ipc_similarity[n].get(company_name, 0)
    #     #     invest_co_ct.at[index, f'peer_tech_net_closeness_{n}'] = cosine_sim[n].loc[company_name].mean() if company_name in cosine_sim[n].index else 0
                    
    #     filter_value = filter_patent[filter_patent.Co==company_name].sort_values(['申请日','公开（公告）日']).drop_duplicates(['申请号','专利类型'],keep='last')
    #     if filter_value.empty:
    #         continue

    #     mask_years = filter_value['申请年'].between(year - knowledge_year, year - 1)
    #     tech_category = filter_value.loc[mask_years,'技术大类'].dropna().to_list()
    #     past_citations = filter_value.loc[mask_years,'引证专利_A'].explode().dropna()#.str.strip()
    #     past_patents = filter_value.loc[filter_value['申请年'] != year,'公开（公告）号'].dropna().str.strip()
    #     existing_knowledge = set(past_patents) | set(past_citations)

    #     filter_value_year = filter_value[filter_value['申请年'] ==  year]
    #     if filter_value_year.empty: 
    #         continue
    #     explorative_percentage = filter_value_year['引证专利_A'].apply(lambda x:len(set(x) - set(existing_knowledge)) / len(x) if (isinstance(x, list) and len(x)) else 0)
    #     exploitative_percentage = filter_value_year['引证专利_A'].apply(lambda x:len(set(x) & set(existing_knowledge)) / len(x) if (isinstance(x, list) and len(x)) else 0)
        
    #     invest_co_ct.at[index, 'explorative'] = (explorative_percentage > 0.8).sum() / len(explorative_percentage) #.mean()
    #     invest_co_ct.at[index, 'exploitative'] = (exploitative_percentage > 0.8).sum() / len(exploitative_percentage)
    #     invest_co_ct.at[index, 'jianjin_patent_num'] = filter_value_year['技术大类'].isin(tech_category).sum()
        
    print(year,filter_patent['Co'].nunique(),filter_invest['企业全称'].nunique())

1992 4 4
1993 5 9
1994 5 4
1995 10 6
1996 17 14
1997 22 14
1998 28 10
1999 42 2
2000 60 27
2001 109 27
2002 163 35
2003 229 36
2004 312 37
2005 416 31
2006 561 45
2007 751 92
2008 1039 114
2009 1332 125


In [33]:
(invest_co_xj['peer_tech_net_closeness_1']==0).sum(),(invest_co_xj['peer_tech_net_closeness_2']==0).sum(),(invest_co_xj['peer_tech_net_closeness_3']==0).sum()

(np.int64(0), np.int64(0), np.int64(0))

In [31]:
invest_co_xj

,行业,子行业,企业全称,交易时间,交易轮次,all,explorative,exploitative,jianjin_patent_num,首次上市日期,...,VC声誉,ChinaVC,GVC,政府引导基金参与,peer_ipc_closeness_1,peer_tech_net_closeness_1,peer_ipc_closeness_2,peer_tech_net_closeness_2,peer_ipc_closeness_3,peer_tech_net_closeness_3
0,先进制造,机器人,北京极智嘉科技股份有限公司,2017-03-19,A+,1,NaN,NaN,NaN,NaT,...,0.00000,0,0,0,NaN,<NA>,NaN,<NA>,NaN,<NA>
1,先进制造,新材料,上海晶璀新材料科技有限责任公司,2017-03-18,天使轮,1,NaN,NaN,NaN,NaT,...,0.00000,1,0,0,NaN,<NA>,NaN,<NA>,NaN,<NA>
2,先进制造,高端装备制造,秦皇岛天秦装备制造股份有限公司,2017-03-17,定向增发,1,0.000000,0.0,2.0,2020-12-25,...,0.00000,1,0,0,1.000000,0.87178,1.000000,0.868349,1.000000,0.869544
3,先进制造,先进制造其他,沈阳上博智像科技有限公司,2017-03-17,Pre-A,1,0.571429,0.0,0.0,NaT,...,NaN,0,0,0,0.857143,0.860341,0.777778,0.867281,0.777778,0.861736
4,先进制造,高端装备制造,深圳天神无人机技术有限公司,2017-03-16,A,1,NaN,NaN,NaN,NaT,...,0.00000,1,0,0,NaN,<NA>,NaN,<NA>,NaN,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27302,先进制造,智能装备,浙江紫明低温科技有限公司,2024-01-16,A轮,0,0.666667,0.0,0.0,NaT,...,0.60206,1,0,0,1.000000,0.841593,1.000000,0.840778,1.000000,0.838551
27303,先进制造,智能装备,浙江紫明低温科技有限公司,2023-05-12,Pre-A轮,0,NaN,NaN,NaN,NaT,...,NaN,0,0,0,NaN,<NA>,NaN,<NA>,NaN,<NA>
27304,先进制造,智能装备,浙江紫明低温科技有限公司,2022-10-21,天使轮,0,NaN,NaN,NaN,NaT,...,0.00000,1,0,0,NaN,<NA>,NaN,<NA>,NaN,<NA>
27305,先进制造,集成电路,安徽骆华生物科技有限公司,2024-03-01,A轮,0,NaN,NaN,NaN,NaT,...,0.30103,0,0,0,NaN,<NA>,1.000000,0.824388,1.000000,0.893035


In [35]:
invest_co_xj.to_excel('data\\invest_patent_先进制造_202410_4.xlsx')

In [38]:
invest_co_xj.子行业.unique()

array(['机器人', '新材料', '高端装备制造', '先进制造其他', '航空航天', '通信制造', '集成电路', '家用电器',
       'AR/VR设备', '可穿戴设备', '传感设备', '智能装备', '电子设备', '无人机', '新能源', '工业4.0',
       '3D打印', '国防军工', '其他工业'], dtype=object)

In [39]:
invest_co_ct.子行业.unique()

array(['机械设备', '木材加工制造', '造纸行业', '传统制造其他', '纺织行业', '电工电气', '金属矿业制造', '环保',
       '轻工制造', '采矿业', '化工产业', '公用事业', '纺织业', '园林园艺', '其他传统制造', '其他工业',
       '集成电路', '数字人', '数字货币'], dtype=object)

In [41]:
invest_co_xj.子行业.isin(['其他工业']).sum()

np.int64(116)

In [42]:
invest_co_xj=invest_co_xj[~invest_co_xj.子行业.isin(['其他工业'])]

In [44]:
invest_co_ct.子行业.isin(['其他工业','数字人', '数字货币','集成电路']).sum()

np.int64(7)

In [45]:
invest_co_ct=invest_co_ct[~invest_co_ct.子行业.isin(['其他工业','数字人', '数字货币','集成电路'])]

In [46]:
invest_co_ct.to_excel('data/invest_patent_传统制造_202410_5.xlsx')
invest_co_xj.to_excel('data/invest_patent_先进制造_202410_5.xlsx')

In [13]:
invest_co_ct=pd.read_excel('data/invest_patent_传统制造_202410_5.xlsx',index_col=0)
invest_co_xj=pd.read_excel('data/invest_patent_先进制造_202410_5.xlsx',index_col=0)

In [47]:
invest_co_xj.columns

Index(['行业', '子行业', '企业全称', '交易时间', '交易轮次', 'all', 'explorative',
       'exploitative', 'jianjin_patent_num', '首次上市日期', '是否上市',
       'tech_net_Explorative1_0.7', 'tech_net_Exploitative1_0.7',
       'tech_net_Explorative1_0.75', 'tech_net_Exploitative1_0.75',
       'tech_net_Explorative1_0.8', 'tech_net_Exploitative1_0.8',
       'tech_net_Explorative1_0.85', 'tech_net_Exploitative1_0.85',
       'tech_net_Explorative2_0.7', 'tech_net_Exploitative2_0.7',
       'tech_net_Explorative2_0.75', 'tech_net_Exploitative2_0.75',
       'tech_net_Explorative2_0.8', 'tech_net_Exploitative2_0.8',
       'tech_net_Explorative2_0.85', 'tech_net_Exploitative2_0.85',
       'tech_net_Explorative3_0.7', 'tech_net_Exploitative3_0.7',
       'tech_net_Explorative3_0.75', 'tech_net_Exploitative3_0.75',
       'tech_net_Explorative3_0.8', 'tech_net_Exploitative3_0.8',
       'tech_net_Explorative3_0.85', 'tech_net_Exploitative3_0.85',
       'tech_net_Explorative4_0.7', 'tech_net_Exploitative4_0.7',
 

In [50]:
ipc_id_xj=pd.read_excel('data\\先进制造专利层面ipc_technet技术相似性_2.xlsx',index_col=0)
ipc_id_xj.columns

Index(['申请年', '技术大类', 'mean', 'var', 'std', 'cv',
       'tech_net_Explorative_mean1_0.45', 'tech_net_Exploitative_mean1_0.65',
       'tech_net_Explorative_median1_0.45',
       'tech_net_Exploitative_median1_0.65', 'tech_net_Explorative_mean1_0.5',
       'tech_net_Exploitative_mean1_0.7', 'tech_net_Explorative_median1_0.5',
       'tech_net_Exploitative_median1_0.7', 'tech_net_Explorative_mean1_0.55',
       'tech_net_Exploitative_mean1_0.75', 'tech_net_Explorative_median1_0.55',
       'tech_net_Exploitative_median1_0.75', 'tech_net_Explorative_mean2_0.45',
       'tech_net_Exploitative_mean2_0.65', 'tech_net_Explorative_median2_0.45',
       'tech_net_Exploitative_median2_0.65', 'tech_net_Explorative_mean2_0.5',
       'tech_net_Exploitative_mean2_0.7', 'tech_net_Explorative_median2_0.5',
       'tech_net_Exploitative_median2_0.7', 'tech_net_Explorative_mean2_0.55',
       'tech_net_Exploitative_mean2_0.75', 'tech_net_Explorative_median2_0.55',
       'tech_net_Exploitative_median

In [52]:
ipc_id_xj.shape,invest_co_xj.shape

((2610, 67), (27191, 77))

In [66]:
invest_co_xj.filter(like='tech_net_Explorative').corr().dropna(how='all',axis=0).dropna(how='all',axis=1).sort_index(axis=0).sort_index(axis=1)

,tech_net_Explorative1_0.7,tech_net_Explorative1_0.75,tech_net_Explorative1_0.8,tech_net_Explorative2_0.7,tech_net_Explorative2_0.75,tech_net_Explorative2_0.8,tech_net_Explorative2_0.85,tech_net_Explorative3_0.7,tech_net_Explorative3_0.75,tech_net_Explorative3_0.8,tech_net_Explorative3_0.85,tech_net_Explorative4_0.7,tech_net_Explorative4_0.75,tech_net_Explorative4_0.8,tech_net_Explorative4_0.85,tech_net_Explorative5_0.7,tech_net_Explorative5_0.75,tech_net_Explorative5_0.8,tech_net_Explorative5_0.85
tech_net_Explorative1_0.7,1.000000,0.633041,0.137726,0.981551,0.628623,0.136746,0.007326,0.969288,0.623507,0.134235,0.007326,0.958160,0.616296,0.130798,0.007326,0.949159,0.611219,0.126830,0.007326
tech_net_Explorative1_0.75,0.633041,1.000000,0.247314,0.634851,0.987145,0.243825,0.016164,0.636773,0.973344,0.238126,0.016164,0.636550,0.962114,0.236151,0.016164,0.636228,0.948831,0.230400,0.016164
tech_net_Explorative1_0.8,0.137726,0.247314,1.000000,0.139203,0.247691,0.949480,0.109133,0.140234,0.251127,0.934797,0.109133,0.140929,0.254751,0.918113,0.109133,0.139643,0.255546,0.905069,0.109133
tech_net_Explorative2_0.7,0.981551,0.634851,0.139203,1.000000,0.630942,0.138219,0.007350,0.987726,0.626723,0.135564,0.007350,0.974992,0.619929,0.132106,0.007350,0.965967,0.615091,0.128177,0.007350
tech_net_Explorative2_0.75,0.628623,0.987145,0.247691,0.630942,1.000000,0.244484,0.016267,0.633265,0.984396,0.238887,0.016267,0.633585,0.972820,0.236854,0.016267,0.633819,0.958660,0.231031,0.016267
tech_net_Explorative2_0.8,0.136746,0.243825,0.949480,0.138219,0.244484,1.000000,0.055091,0.139169,0.248135,0.979768,0.055091,0.139890,0.252280,0.957799,0.055091,0.140284,0.253272,0.945318,0.055091
tech_net_Explorative2_0.85,0.007326,0.016164,0.109133,0.007350,0.016267,0.055091,1.000000,0.007386,0.016441,0.113909,1.000000,0.007421,0.016686,0.115390,1.000000,0.007459,0.016928,0.117232,1.000000
tech_net_Explorative3_0.7,0.969288,0.636773,0.140234,0.987726,0.633265,0.139169,0.007386,1.000000,0.629480,0.136874,0.007386,0.987335,0.623177,0.133594,0.007386,0.977246,0.618842,0.129606,0.007386
tech_net_Explorative3_0.75,0.623507,0.973344,0.251127,0.626723,0.984396,0.248135,0.016441,0.629480,1.000000,0.243085,0.016441,0.630323,0.986898,0.240456,0.016441,0.631087,0.973603,0.234705,0.016441
tech_net_Explorative3_0.8,0.134235,0.238126,0.934797,0.135564,0.238887,0.979768,0.113909,0.136874,0.243085,1.000000,0.113909,0.137562,0.247448,0.975171,0.113909,0.138041,0.248787,0.961982,0.113909


In [68]:
# invest_co_xj.filter(like='tech_net_Exploitative').corr().dropna(how='all',axis=0).dropna(how='all',axis=1).sort_index(axis=0).sort_index(axis=1)

In [78]:
(invest_co_xj.filter(like='tech_net_Explorative')==0).sum().sort_index(axis=0)

tech_net_Explorative1_0.7     4658
tech_net_Explorative1_0.75    1192
tech_net_Explorative1_0.8       34
tech_net_Explorative1_0.85       0
tech_net_Explorative2_0.7     4640
tech_net_Explorative2_0.75    1189
tech_net_Explorative2_0.8       30
tech_net_Explorative2_0.85       0
tech_net_Explorative3_0.7     4597
tech_net_Explorative3_0.75    1169
tech_net_Explorative3_0.8       30
tech_net_Explorative3_0.85       0
tech_net_Explorative4_0.7     4565
tech_net_Explorative4_0.75    1137
tech_net_Explorative4_0.8       30
tech_net_Explorative4_0.85       0
tech_net_Explorative5_0.7     4522
tech_net_Explorative5_0.75    1098
tech_net_Explorative5_0.8       30
tech_net_Explorative5_0.85       0
dtype: int64

In [8]:
(invest_co_xj.filter(like='tech_net_Explorative')==1).sum().sort_index(axis=0)

tech_net_Explorative1_0.7       860
tech_net_Explorative1_0.75     2637
tech_net_Explorative1_0.8     10440
tech_net_Explorative1_0.85    13171
tech_net_Explorative2_0.7       865
tech_net_Explorative2_0.75     2670
tech_net_Explorative2_0.8     10507
tech_net_Explorative2_0.85    13170
tech_net_Explorative3_0.7       873
tech_net_Explorative3_0.75     2708
tech_net_Explorative3_0.8     10574
tech_net_Explorative3_0.85    13170
tech_net_Explorative4_0.7       872
tech_net_Explorative4_0.75     2742
tech_net_Explorative4_0.8     10663
tech_net_Explorative4_0.85    13170
tech_net_Explorative5_0.7       869
tech_net_Explorative5_0.75     2757
tech_net_Explorative5_0.8     10784
tech_net_Explorative5_0.85    13170
dtype: int64

In [79]:
(invest_co_xj.filter(like='tech_net_Exploitative')==0).sum().sort_index(axis=0)

tech_net_Exploitative1_0.7       860
tech_net_Exploitative1_0.75     2637
tech_net_Exploitative1_0.8     10440
tech_net_Exploitative1_0.85    13171
tech_net_Exploitative2_0.7       865
tech_net_Exploitative2_0.75     2670
tech_net_Exploitative2_0.8     10507
tech_net_Exploitative2_0.85    13170
tech_net_Exploitative3_0.7       873
tech_net_Exploitative3_0.75     2708
tech_net_Exploitative3_0.8     10574
tech_net_Exploitative3_0.85    13170
tech_net_Exploitative4_0.7       872
tech_net_Exploitative4_0.75     2742
tech_net_Exploitative4_0.8     10663
tech_net_Exploitative4_0.85    13170
tech_net_Exploitative5_0.7       869
tech_net_Exploitative5_0.75     2757
tech_net_Exploitative5_0.8     10784
tech_net_Exploitative5_0.85    13170
dtype: int64

In [7]:
(invest_co_xj.filter(like='tech_net_Exploitative')==1).sum().sort_index(axis=0)

tech_net_Exploitative1_0.7     4658
tech_net_Exploitative1_0.75    1192
tech_net_Exploitative1_0.8       34
tech_net_Exploitative1_0.85       0
tech_net_Exploitative2_0.7     4640
tech_net_Exploitative2_0.75    1189
tech_net_Exploitative2_0.8       30
tech_net_Exploitative2_0.85       0
tech_net_Exploitative3_0.7     4597
tech_net_Exploitative3_0.75    1169
tech_net_Exploitative3_0.8       30
tech_net_Exploitative3_0.85       0
tech_net_Exploitative4_0.7     4565
tech_net_Exploitative4_0.75    1137
tech_net_Exploitative4_0.8       30
tech_net_Exploitative4_0.85       0
tech_net_Exploitative5_0.7     4522
tech_net_Exploitative5_0.75    1098
tech_net_Exploitative5_0.8       30
tech_net_Exploitative5_0.85       0
dtype: int64

In [93]:
(invest_co_xj.filter(like='tech_net_Explorative')>0).sum().add((invest_co_xj.filter(like='tech_net_Exploitative')>0).sum(), fill_value=0).sort_index(axis=0)

tech_net_Exploitative1_0.7     12311.0
tech_net_Exploitative1_0.75    10534.0
tech_net_Exploitative1_0.8      2731.0
tech_net_Exploitative1_0.85        0.0
tech_net_Exploitative2_0.7     12306.0
tech_net_Exploitative2_0.75    10501.0
tech_net_Exploitative2_0.8      2664.0
tech_net_Exploitative2_0.85        1.0
tech_net_Exploitative3_0.7     12298.0
tech_net_Exploitative3_0.75    10463.0
tech_net_Exploitative3_0.8      2597.0
tech_net_Exploitative3_0.85        1.0
tech_net_Exploitative4_0.7     12299.0
tech_net_Exploitative4_0.75    10429.0
tech_net_Exploitative4_0.8      2508.0
tech_net_Exploitative4_0.85        1.0
tech_net_Exploitative5_0.7     12302.0
tech_net_Exploitative5_0.75    10414.0
tech_net_Exploitative5_0.8      2387.0
tech_net_Exploitative5_0.85        1.0
tech_net_Explorative1_0.7       8513.0
tech_net_Explorative1_0.75     11979.0
tech_net_Explorative1_0.8      13137.0
tech_net_Explorative1_0.85     13171.0
tech_net_Explorative2_0.7       8531.0
tech_net_Explorative2_0.7

In [95]:
a=(invest_co_xj.filter(like='tech_net_Explorative')>0).sum().sort_index(axis=0)
b=(invest_co_xj.filter(like='tech_net_Exploitative')>0).sum().sort_index(axis=0)
total =a.values+b.values
total

array([20824, 22513, 15868, 13171, 20837, 22483, 15805, 13172, 20872,
       22465, 15738, 13172, 20905, 22463, 15649, 13172, 20951, 22487,
       15528, 13172])

In [99]:
sorted_indices = total.argsort()
sorted_total = total[sorted_indices]
for idxa,idxb, val in zip(a.index[sorted_indices], b.index[sorted_indices],sorted_total):
    print(f"{idxa},{idxb}: {val}")

tech_net_Explorative1_0.85,tech_net_Exploitative1_0.85: 13171
tech_net_Explorative2_0.85,tech_net_Exploitative2_0.85: 13172
tech_net_Explorative4_0.85,tech_net_Exploitative4_0.85: 13172
tech_net_Explorative3_0.85,tech_net_Exploitative3_0.85: 13172
tech_net_Explorative5_0.85,tech_net_Exploitative5_0.85: 13172
tech_net_Explorative5_0.8,tech_net_Exploitative5_0.8: 15528
tech_net_Explorative4_0.8,tech_net_Exploitative4_0.8: 15649
tech_net_Explorative3_0.8,tech_net_Exploitative3_0.8: 15738
tech_net_Explorative2_0.8,tech_net_Exploitative2_0.8: 15805
tech_net_Explorative1_0.8,tech_net_Exploitative1_0.8: 15868
tech_net_Explorative1_0.7,tech_net_Exploitative1_0.7: 20824
tech_net_Explorative2_0.7,tech_net_Exploitative2_0.7: 20837
tech_net_Explorative3_0.7,tech_net_Exploitative3_0.7: 20872
tech_net_Explorative4_0.7,tech_net_Exploitative4_0.7: 20905
tech_net_Explorative5_0.7,tech_net_Exploitative5_0.7: 20951
tech_net_Explorative4_0.75,tech_net_Exploitative4_0.75: 22463
tech_net_Explorative3_0.75,t

In [43]:
invest_co_xj.filter(like='1_0.7')

,tech_net_Explorative1_0.7,tech_net_Exploitative1_0.7,tech_net_Explorative1_0.75,tech_net_Exploitative1_0.75
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,0.000000,1.000000,0.000000,1.000000
3,0.142857,0.857143,0.428571,0.571429
4,NaN,NaN,NaN,NaN
...,...,...,...,...
27186,0.000000,1.000000,0.333333,0.666667
27187,NaN,NaN,NaN,NaN
27188,NaN,NaN,NaN,NaN
27189,NaN,NaN,NaN,NaN


In [45]:
invest_co_xj.filter(like='3_0.8')

,tech_net_Explorative3_0.8,tech_net_Exploitative3_0.8,tech_net_Explorative3_0.85,tech_net_Exploitative3_0.85
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,1.0,0.0,1.0,0.0
3,1.0,0.0,1.0,0.0
4,NaN,NaN,NaN,NaN
...,...,...,...,...
27186,1.0,0.0,1.0,0.0
27187,NaN,NaN,NaN,NaN
27188,NaN,NaN,NaN,NaN
27189,NaN,NaN,NaN,NaN


In [10]:
a=invest_co_xj.filter(like='tech_net_Explorative')
b=invest_co_xj.filter(like='tech_net_Exploitative')
((a>0)&(a<1)).sum()

tech_net_Explorative1_0.7     7653
tech_net_Explorative1_0.75    9342
tech_net_Explorative1_0.8     2697
tech_net_Explorative1_0.85       0
tech_net_Explorative2_0.7     7666
tech_net_Explorative2_0.75    9312
tech_net_Explorative2_0.8     2634
tech_net_Explorative2_0.85       1
tech_net_Explorative3_0.7     7701
tech_net_Explorative3_0.75    9294
tech_net_Explorative3_0.8     2567
tech_net_Explorative3_0.85       1
tech_net_Explorative4_0.7     7734
tech_net_Explorative4_0.75    9292
tech_net_Explorative4_0.8     2478
tech_net_Explorative4_0.85       1
tech_net_Explorative5_0.7     7780
tech_net_Explorative5_0.75    9316
tech_net_Explorative5_0.8     2357
tech_net_Explorative5_0.85       1
dtype: int64

In [36]:
ia=((a>0)&(a<1)).sum().sort_index(axis=0)
ib=((b>0)&(b<1)).sum().sort_index(axis=0)
ia

tech_net_Explorative1_0.7     7653
tech_net_Explorative1_0.75    9342
tech_net_Explorative1_0.8     2697
tech_net_Explorative1_0.85       0
tech_net_Explorative2_0.7     7666
tech_net_Explorative2_0.75    9312
tech_net_Explorative2_0.8     2634
tech_net_Explorative2_0.85       1
tech_net_Explorative3_0.7     7701
tech_net_Explorative3_0.75    9294
tech_net_Explorative3_0.8     2567
tech_net_Explorative3_0.85       1
tech_net_Explorative4_0.7     7734
tech_net_Explorative4_0.75    9292
tech_net_Explorative4_0.8     2478
tech_net_Explorative4_0.85       1
tech_net_Explorative5_0.7     7780
tech_net_Explorative5_0.75    9316
tech_net_Explorative5_0.8     2357
tech_net_Explorative5_0.85       1
dtype: int64

In [38]:
ib

tech_net_Exploitative1_0.7     7653
tech_net_Exploitative1_0.75    9342
tech_net_Exploitative1_0.8     2697
tech_net_Exploitative1_0.85       0
tech_net_Exploitative2_0.7     7666
tech_net_Exploitative2_0.75    9312
tech_net_Exploitative2_0.8     2634
tech_net_Exploitative2_0.85       1
tech_net_Exploitative3_0.7     7701
tech_net_Exploitative3_0.75    9294
tech_net_Exploitative3_0.8     2567
tech_net_Exploitative3_0.85       1
tech_net_Exploitative4_0.7     7734
tech_net_Exploitative4_0.75    9292
tech_net_Exploitative4_0.8     2478
tech_net_Exploitative4_0.85       1
tech_net_Exploitative5_0.7     7780
tech_net_Exploitative5_0.75    9316
tech_net_Exploitative5_0.8     2357
tech_net_Exploitative5_0.85       1
dtype: int64

In [40]:
total =(ia.values+ib.values)/2
sorted_indices = total.argsort()
for idxa,idxb, val in zip(ia.index[sorted_indices], ib.index[sorted_indices],total[sorted_indices]):
    print(f"{idxa},{idxb}: {val}")

tech_net_Explorative1_0.85,tech_net_Exploitative1_0.85: 0.0
tech_net_Explorative2_0.85,tech_net_Exploitative2_0.85: 1.0
tech_net_Explorative4_0.85,tech_net_Exploitative4_0.85: 1.0
tech_net_Explorative3_0.85,tech_net_Exploitative3_0.85: 1.0
tech_net_Explorative5_0.85,tech_net_Exploitative5_0.85: 1.0
tech_net_Explorative5_0.8,tech_net_Exploitative5_0.8: 2357.0
tech_net_Explorative4_0.8,tech_net_Exploitative4_0.8: 2478.0
tech_net_Explorative3_0.8,tech_net_Exploitative3_0.8: 2567.0
tech_net_Explorative2_0.8,tech_net_Exploitative2_0.8: 2634.0
tech_net_Explorative1_0.8,tech_net_Exploitative1_0.8: 2697.0
tech_net_Explorative1_0.7,tech_net_Exploitative1_0.7: 7653.0
tech_net_Explorative2_0.7,tech_net_Exploitative2_0.7: 7666.0
tech_net_Explorative3_0.7,tech_net_Exploitative3_0.7: 7701.0
tech_net_Explorative4_0.7,tech_net_Exploitative4_0.7: 7734.0
tech_net_Explorative5_0.7,tech_net_Exploitative5_0.7: 7780.0
tech_net_Explorative4_0.75,tech_net_Exploitative4_0.75: 9292.0
tech_net_Explorative3_0.75,

In [29]:
sa=a[(a>0)&(a<1)].std().sort_index(axis=0)
sb=b[(b>0)&(b<1)].std().sort_index(axis=0)
sb

tech_net_Exploitative1_0.7     0.219646
tech_net_Exploitative1_0.75    0.214609
tech_net_Exploitative1_0.8     0.101830
tech_net_Exploitative1_0.85         NaN
tech_net_Exploitative2_0.7     0.219451
tech_net_Exploitative2_0.75    0.213766
tech_net_Exploitative2_0.8     0.102203
tech_net_Exploitative2_0.85         NaN
tech_net_Exploitative3_0.7     0.219571
tech_net_Exploitative3_0.75    0.213274
tech_net_Exploitative3_0.8     0.101762
tech_net_Exploitative3_0.85         NaN
tech_net_Exploitative4_0.7     0.219820
tech_net_Exploitative4_0.75    0.212864
tech_net_Exploitative4_0.8     0.102027
tech_net_Exploitative4_0.85         NaN
tech_net_Exploitative5_0.7     0.219886
tech_net_Exploitative5_0.75    0.212608
tech_net_Exploitative5_0.8     0.102006
tech_net_Exploitative5_0.85         NaN
dtype: float64

In [35]:
sa

tech_net_Explorative1_0.7     0.219646
tech_net_Explorative1_0.75    0.214609
tech_net_Explorative1_0.8     0.101830
tech_net_Explorative1_0.85         NaN
tech_net_Explorative2_0.7     0.219451
tech_net_Explorative2_0.75    0.213766
tech_net_Explorative2_0.8     0.102203
tech_net_Explorative2_0.85         NaN
tech_net_Explorative3_0.7     0.219571
tech_net_Explorative3_0.75    0.213274
tech_net_Explorative3_0.8     0.101762
tech_net_Explorative3_0.85         NaN
tech_net_Explorative4_0.7     0.219820
tech_net_Explorative4_0.75    0.212864
tech_net_Explorative4_0.8     0.102027
tech_net_Explorative4_0.85         NaN
tech_net_Explorative5_0.7     0.219886
tech_net_Explorative5_0.75    0.212608
tech_net_Explorative5_0.8     0.102006
tech_net_Explorative5_0.85         NaN
dtype: float64

In [46]:
total =(sa.values+sb.values)/2
total

array([0.2196459 , 0.21460891, 0.10183044,        nan, 0.21945146,
       0.21376569, 0.10220309,        nan, 0.21957106, 0.21327401,
       0.10176177,        nan, 0.21981992, 0.21286351, 0.10202688,
              nan, 0.21988595, 0.21260804, 0.10200622,        nan])

In [47]:
sorted_indices = total.argsort()
for idxa,idxb, val in zip(sa.index[sorted_indices], sb.index[sorted_indices],total[sorted_indices]):
    print(f"std {idxa},{idxb}: {val}")

std tech_net_Explorative3_0.8,tech_net_Exploitative3_0.8: 0.10176177436965603
std tech_net_Explorative1_0.8,tech_net_Exploitative1_0.8: 0.10183043978509285
std tech_net_Explorative5_0.8,tech_net_Exploitative5_0.8: 0.10200621806910444
std tech_net_Explorative4_0.8,tech_net_Exploitative4_0.8: 0.10202687836028274
std tech_net_Explorative2_0.8,tech_net_Exploitative2_0.8: 0.10220308722911944
std tech_net_Explorative5_0.75,tech_net_Exploitative5_0.75: 0.21260803688167582
std tech_net_Explorative4_0.75,tech_net_Exploitative4_0.75: 0.21286351111723187
std tech_net_Explorative3_0.75,tech_net_Exploitative3_0.75: 0.21327401329195256
std tech_net_Explorative2_0.75,tech_net_Exploitative2_0.75: 0.2137656862217931
std tech_net_Explorative1_0.75,tech_net_Exploitative1_0.75: 0.2146089118234436
std tech_net_Explorative2_0.7,tech_net_Exploitative2_0.7: 0.21945146318466385
std tech_net_Explorative3_0.7,tech_net_Exploitative3_0.7: 0.21957106148507616
std tech_net_Explorative1_0.7,tech_net_Exploitative1_0.7

In [48]:
a=invest_co_ct.filter(like='tech_net_Explorative')
b=invest_co_ct.filter(like='tech_net_Exploitative')
ia=((a>0)&(a<1)).sum().sort_index(axis=0)
ib=((b>0)&(b<1)).sum().sort_index(axis=0)
total =(ia.values+ib.values)/2
sorted_indices = total.argsort()
for idxa,idxb, val in zip(ia.index[sorted_indices], ib.index[sorted_indices],total[sorted_indices]):
    print(f"{idxa},{idxb}: {val}")

tech_net_Explorative1_0.85,tech_net_Exploitative1_0.85: 0.0
tech_net_Explorative2_0.85,tech_net_Exploitative2_0.85: 0.0
tech_net_Explorative4_0.85,tech_net_Exploitative4_0.85: 0.0
tech_net_Explorative3_0.85,tech_net_Exploitative3_0.85: 0.0
tech_net_Explorative5_0.85,tech_net_Exploitative5_0.85: 0.0
tech_net_Explorative5_0.8,tech_net_Exploitative5_0.8: 3020.0
tech_net_Explorative4_0.8,tech_net_Exploitative4_0.8: 3073.0
tech_net_Explorative3_0.8,tech_net_Exploitative3_0.8: 3140.0
tech_net_Explorative2_0.8,tech_net_Exploitative2_0.8: 3182.0
tech_net_Explorative1_0.8,tech_net_Exploitative1_0.8: 3222.0
tech_net_Explorative4_0.7,tech_net_Exploitative4_0.7: 3629.0
tech_net_Explorative1_0.7,tech_net_Exploitative1_0.7: 3631.0
tech_net_Explorative5_0.7,tech_net_Exploitative5_0.7: 3631.0
tech_net_Explorative2_0.7,tech_net_Exploitative2_0.7: 3633.0
tech_net_Explorative3_0.7,tech_net_Exploitative3_0.7: 3636.0
tech_net_Explorative1_0.75,tech_net_Exploitative1_0.75: 5946.0
tech_net_Explorative2_0.75,

In [49]:
sa=a[(a>0)&(a<1)].std().sort_index(axis=0)
sb=b[(b>0)&(b<1)].std().sort_index(axis=0)
total =(sa.values+sb.values)/2
sorted_indices = total.argsort()
for idxa,idxb, val in zip(sa.index[sorted_indices], sb.index[sorted_indices],total[sorted_indices]):
    print(f"std {idxa},{idxb}: {val}")

std tech_net_Explorative5_0.8,tech_net_Exploitative5_0.8: 0.1280626946439684
std tech_net_Explorative4_0.8,tech_net_Exploitative4_0.8: 0.12943335564794095
std tech_net_Explorative3_0.8,tech_net_Exploitative3_0.8: 0.13174552393590874
std tech_net_Explorative2_0.8,tech_net_Exploitative2_0.8: 0.13268992837450644
std tech_net_Explorative1_0.8,tech_net_Exploitative1_0.8: 0.1335072822488253
std tech_net_Explorative1_0.7,tech_net_Exploitative1_0.7: 0.21398441190603484
std tech_net_Explorative3_0.7,tech_net_Exploitative3_0.7: 0.2149683119275638
std tech_net_Explorative5_0.7,tech_net_Exploitative5_0.7: 0.21511338326052146
std tech_net_Explorative2_0.7,tech_net_Exploitative2_0.7: 0.21544053622962733
std tech_net_Explorative4_0.7,tech_net_Exploitative4_0.7: 0.21552747667765945
std tech_net_Explorative5_0.75,tech_net_Exploitative5_0.75: 0.22057755190433118
std tech_net_Explorative4_0.75,tech_net_Exploitative4_0.75: 0.2208533553412621
std tech_net_Explorative3_0.75,tech_net_Exploitative3_0.75: 0.22

In [53]:
a[(a>0)&(a<1)].mean()

tech_net_Explorative1_0.7     0.295711
tech_net_Explorative1_0.75    0.415498
tech_net_Explorative1_0.8     0.832797
tech_net_Explorative1_0.85         NaN
tech_net_Explorative2_0.7     0.296454
tech_net_Explorative2_0.75    0.415777
tech_net_Explorative2_0.8     0.836348
tech_net_Explorative2_0.85         NaN
tech_net_Explorative3_0.7     0.295868
tech_net_Explorative3_0.75    0.417503
tech_net_Explorative3_0.8     0.838102
tech_net_Explorative3_0.85         NaN
tech_net_Explorative4_0.7     0.296427
tech_net_Explorative4_0.75    0.419262
tech_net_Explorative4_0.8     0.840417
tech_net_Explorative4_0.85         NaN
tech_net_Explorative5_0.7     0.295789
tech_net_Explorative5_0.75    0.420529
tech_net_Explorative5_0.8     0.843244
tech_net_Explorative5_0.85         NaN
dtype: float64

In [69]:
ipc_id_xj.filter(like='tech_net_Explorative_mean').corr().dropna(how='all',axis=0).dropna(how='all',axis=1).sort_index(axis=0).sort_index(axis=1)

,tech_net_Explorative_mean1_0.55,tech_net_Explorative_mean2_0.55,tech_net_Explorative_mean3_0.55,tech_net_Explorative_mean4_0.55,tech_net_Explorative_mean5_0.55
tech_net_Explorative_mean1_0.55,1.000000,0.779042,0.662925,0.735509,0.734118
tech_net_Explorative_mean2_0.55,0.779042,1.000000,0.982085,0.989592,0.987721
tech_net_Explorative_mean3_0.55,0.662925,0.982085,1.000000,0.992987,0.991106
tech_net_Explorative_mean4_0.55,0.735509,0.989592,0.992987,1.000000,0.998103
tech_net_Explorative_mean5_0.55,0.734118,0.987721,0.991106,0.998103,1.000000


In [70]:
ipc_id_xj.filter(like='tech_net_Exploitative_mean').corr().dropna(how='all',axis=0).dropna(how='all',axis=1).sort_index(axis=0).sort_index(axis=1)

,tech_net_Exploitative_mean1_0.65,tech_net_Exploitative_mean1_0.7,tech_net_Exploitative_mean1_0.75,tech_net_Exploitative_mean2_0.65,tech_net_Exploitative_mean2_0.7,tech_net_Exploitative_mean2_0.75,tech_net_Exploitative_mean3_0.65,tech_net_Exploitative_mean3_0.7,tech_net_Exploitative_mean3_0.75,tech_net_Exploitative_mean4_0.65,tech_net_Exploitative_mean4_0.7,tech_net_Exploitative_mean4_0.75,tech_net_Exploitative_mean5_0.65,tech_net_Exploitative_mean5_0.7,tech_net_Exploitative_mean5_0.75
tech_net_Exploitative_mean1_0.65,1.000000,0.697065,0.445120,0.963012,0.690158,0.440763,0.927461,0.687853,0.436569,0.907952,0.682499,0.429738,0.887954,0.676468,0.423282
tech_net_Exploitative_mean1_0.7,0.697065,1.000000,0.695640,0.697583,0.981297,0.691056,0.690429,0.964400,0.684801,0.688175,0.942989,0.673319,0.681691,0.926474,0.663808
tech_net_Exploitative_mean1_0.75,0.445120,0.695640,1.000000,0.448501,0.697420,0.961917,0.449422,0.699162,0.948530,0.449795,0.697187,0.925607,0.450678,0.692827,0.905730
tech_net_Exploitative_mean2_0.65,0.963012,0.697583,0.448501,1.000000,0.694847,0.444812,0.963834,0.694025,0.441149,0.945444,0.690046,0.433890,0.912235,0.683122,0.428010
tech_net_Exploitative_mean2_0.7,0.690158,0.981297,0.697420,0.694847,1.000000,0.696545,0.689641,0.983861,0.693928,0.688868,0.964816,0.682664,0.683794,0.950990,0.672947
tech_net_Exploitative_mean2_0.75,0.440763,0.691056,0.961917,0.444812,0.696545,1.000000,0.446236,0.700425,0.980249,0.447721,0.700025,0.957413,0.449553,0.697510,0.939456
tech_net_Exploitative_mean3_0.65,0.927461,0.690429,0.449422,0.963834,0.689641,0.446236,1.000000,0.691477,0.443553,0.984924,0.689330,0.436679,0.952343,0.685465,0.431070
tech_net_Exploitative_mean3_0.7,0.687853,0.964400,0.699162,0.694025,0.983861,0.700425,0.691477,1.000000,0.699681,0.692240,0.984846,0.690471,0.688391,0.973861,0.682721
tech_net_Exploitative_mean3_0.75,0.436569,0.684801,0.948530,0.441149,0.693928,0.980249,0.443553,0.699681,1.000000,0.445856,0.701619,0.971284,0.448234,0.700825,0.955383
tech_net_Exploitative_mean4_0.65,0.907952,0.688175,0.449795,0.945444,0.688868,0.447721,0.984924,0.692240,0.445856,1.000000,0.692205,0.439360,0.969204,0.690162,0.433986


In [71]:
ipc_id_xj.filter(like='tech_net_Explorative_median').corr().dropna(how='all',axis=0).dropna(how='all',axis=1).sort_index(axis=0).sort_index(axis=1)

,tech_net_Explorative_median1_0.5,tech_net_Explorative_median1_0.55,tech_net_Explorative_median2_0.5,tech_net_Explorative_median2_0.55,tech_net_Explorative_median3_0.5,tech_net_Explorative_median3_0.55,tech_net_Explorative_median4_0.5,tech_net_Explorative_median4_0.55,tech_net_Explorative_median5_0.5,tech_net_Explorative_median5_0.55
tech_net_Explorative_median1_0.5,1.000000,0.033219,0.826910,0.034400,-0.000385,0.034663,-0.000385,0.034563,-0.000385,0.034596
tech_net_Explorative_median1_0.55,0.033219,1.000000,0.034492,0.943568,0.012477,0.803372,0.012477,0.783649,0.012477,0.758792
tech_net_Explorative_median2_0.5,0.826910,0.034492,1.000000,0.039959,0.562017,0.042127,0.562017,0.042035,0.562017,0.043012
tech_net_Explorative_median2_0.55,0.034400,0.943568,0.039959,1.000000,0.020448,0.856223,0.020448,0.835637,0.020448,0.810522
tech_net_Explorative_median3_0.5,-0.000385,0.012477,0.562017,0.020448,1.000000,0.023917,1.000000,0.023903,1.000000,0.025590
tech_net_Explorative_median3_0.55,0.034663,0.803372,0.042127,0.856223,0.023917,1.000000,0.023917,0.979040,0.023917,0.953960
tech_net_Explorative_median4_0.5,-0.000385,0.012477,0.562017,0.020448,1.000000,0.023917,1.000000,0.023903,1.000000,0.025590
tech_net_Explorative_median4_0.55,0.034563,0.783649,0.042035,0.835637,0.023903,0.979040,0.023903,1.000000,0.023903,0.976295
tech_net_Explorative_median5_0.5,-0.000385,0.012477,0.562017,0.020448,1.000000,0.023917,1.000000,0.023903,1.000000,0.025590
tech_net_Explorative_median5_0.55,0.034596,0.758792,0.043012,0.810522,0.025590,0.953960,0.025590,0.976295,0.025590,1.000000


In [62]:
ipc_id_xj.filter(like='tech_net_Exploitative_median').corr().dropna(how='all',axis=0).dropna(how='all',axis=1)

,tech_net_Exploitative_median1_0.65,tech_net_Exploitative_median1_0.7,tech_net_Exploitative_median1_0.75,tech_net_Exploitative_median2_0.65,tech_net_Exploitative_median2_0.7,tech_net_Exploitative_median2_0.75,tech_net_Exploitative_median3_0.65,tech_net_Exploitative_median3_0.7,tech_net_Exploitative_median3_0.75,tech_net_Exploitative_median4_0.65,tech_net_Exploitative_median4_0.7,tech_net_Exploitative_median4_0.75,tech_net_Exploitative_median5_0.65,tech_net_Exploitative_median5_0.7,tech_net_Exploitative_median5_0.75
tech_net_Exploitative_median1_0.65,1.000000,0.805976,0.601505,0.978622,0.804157,0.597416,0.957396,0.791662,0.592494,0.938856,0.781702,0.580992,0.913847,0.766768,0.570322
tech_net_Exploitative_median1_0.7,0.805976,1.000000,0.782944,0.810585,0.972768,0.778978,0.809245,0.957418,0.772461,0.803905,0.939586,0.750194,0.793445,0.917924,0.739938
tech_net_Exploitative_median1_0.75,0.601505,0.782944,1.000000,0.610101,0.777161,0.955791,0.614880,0.779674,0.929690,0.616269,0.774039,0.899645,0.612340,0.761836,0.876600
tech_net_Exploitative_median2_0.65,0.978622,0.810585,0.610101,1.000000,0.814363,0.610399,0.982034,0.804944,0.605696,0.967048,0.797188,0.593896,0.944751,0.783461,0.583752
tech_net_Exploitative_median2_0.7,0.804157,0.972768,0.777161,0.814363,1.000000,0.785544,0.817497,0.983642,0.784806,0.815553,0.968944,0.769902,0.809268,0.950875,0.758824
tech_net_Exploitative_median2_0.75,0.597416,0.778978,0.955791,0.610399,0.785544,1.000000,0.617700,0.792595,0.979182,0.621190,0.790833,0.947510,0.620241,0.783881,0.927862
tech_net_Exploitative_median3_0.65,0.957396,0.809245,0.614880,0.982034,0.817497,0.617700,1.000000,0.811371,0.614960,0.988099,0.806144,0.604076,0.968092,0.798970,0.594674
tech_net_Exploitative_median3_0.7,0.791662,0.957418,0.779674,0.804944,0.983642,0.792595,0.811371,1.000000,0.795184,0.811878,0.990745,0.785181,0.808122,0.975966,0.777029
tech_net_Exploitative_median3_0.75,0.592494,0.772461,0.929690,0.605696,0.784806,0.979182,0.614960,0.795184,1.000000,0.620397,0.797255,0.975573,0.621104,0.793985,0.960117
tech_net_Exploitative_median4_0.65,0.938856,0.803905,0.616269,0.967048,0.815553,0.621190,0.988099,0.811878,0.620397,1.000000,0.809034,0.611318,0.984738,0.803538,0.602867


In [8]:
doc_lda_w2v_xj=pd.read_parquet(f'data\\documents_vec_invest_mfg_xj.parquet')
doc_lda_w2v_xj.shape

(1410528, 300)

In [36]:
doc_vec_xj=doc_lda_w2v_xj.groupby(doc_lda_w2v_xj.index).mean()    #合并复索引的专利vec
vectors = doc_vec_xj.values.astype(np.float32) #np.stack(
faiss.normalize_L2(vectors)#归一化向量，使点积 = 余弦相似度
index_xj = faiss.IndexFlatIP(vectors.shape[1])# faiss.IndexFlatL2
index_xj.add(vectors)
doc_vec_xj.shape

(1389970, 300)

In [39]:
#public_ids = doc_lda_w2v_xj.index.to_list()
public_id_to_faiss_idx_xj = {pid: idx for idx, pid in enumerate(doc_vec_xj.index)}
faiss.write_index(index_xj, 'data\\doc_vec_invest_mfg_xj.faiss')#faiss.read_index

In [ ]:
import pickle
with open('data\\doc_vec_invest_mfg_xj_public_id_to_faiss.pkl', 'wb') as f:
    pickle.dump(public_id_to_faiss_idx_xj, f)

In [8]:
import pickle
index_xj=faiss.read_index('data\\doc_vec_invest_mfg_xj.faiss')
with open('data\\doc_vec_invest_mfg_xj_public_id_to_faiss.pkl', 'rb') as f:
    public_id_to_faiss_idx_xj=pickle.load(f)

In [28]:
# distances, indices = index_xj.search(query_vector,k=2)

In [9]:
index_xj.d,len(public_id_to_faiss_idx_xj)

(300, 1389970)

In [10]:
def batch_faiss_cos_sim(ids1, ids2, index, public_id_to_faiss_idx):
    # 先把公开号转换成 FAISS 内部的索引
    # faiss_idx1 = np.array([public_id_to_faiss_idx[pid] for pid in ids1 if pid in public_id_to_faiss_idx])
    faiss_idx1 = [public_id_to_faiss_idx.get(pid, -1) for pid in ids1]
    faiss_idx2 = [public_id_to_faiss_idx.get(pid, -1) for pid in ids2]

    faiss_idx1 = np.array([idx for idx in faiss_idx1 if idx != -1])
    faiss_idx2 = np.array([idx for idx in faiss_idx2 if idx != -1])

    # 从 FAISS 直接取出向量
    # vecs1 = np.array([index.reconstruct(i) for i in faiss_idx1]).astype(np.float32)
    # vecs2 = np.array([index.reconstruct(i) for i in faiss_idx2]).astype(np.float32)#index.reconstruct_n(
    vecs1 = np.zeros((len(faiss_idx1), index.d), dtype=np.float32)
    vecs2 = np.zeros((len(faiss_idx2), index.d), dtype=np.float32)
    
    index.reconstruct_batch(faiss_idx1, vecs1)
    index.reconstruct_batch(faiss_idx2, vecs2)

    # 计算余弦相似度（点积）
    sim_matrix = np.dot(vecs1, vecs2.T) #np.sum(vecs1 * vecs2, axis=1),np.einsum('ij,ij->i', vecs1, vecs2) 

    return sim_matrix

In [ ]:
%%time
knowledge_year=5
lookup_year=[1,2,3,4,5]
thresholds=[0.7,0.75,0.8,0.85]

for year in sorted(invest_co_ct.交易时间.dt.year.unique()):
    # max_date=pd.to_datetime(year, format='%Y')+ pd.offsets.YearEnd()
    filter_invest = invest_co_ct[invest_co_ct.交易时间.dt.year==year]
    filter_patent = patent_co[(patent_co.传统制造==1)&(patent_co.申请年 <= year)]# <= data
    
    if filter_patent.empty:
        continue

    
    base_tech_net={}
    for n in lookup_year:
        #mask_years = filter_patent.申请日 >  max_date-pd.Timedelta(days=n*365) # pd.Timestamp(year) - pd.DateOffset(years=n)date- pd.DateOffset(years=3)
        mask_years =  filter_patent['申请年'].between(year - n, year - 1)

        base_tech_net[n]= doc_lda_w2v_ct.loc[doc_lda_w2v_ct.index.isin(filter_patent.loc[mask_years,'公开（公告）号'])]

     
    for index,row in filter_invest.iterrows():
        company_name = row['企业全称']
        date = row['交易时间']

        filtered_value_year = filter_patent[(filter_patent.Co==company_name)&(filter_patent['申请年'] ==  year)].sort_values(['申请日','公开（公告）日']).drop_duplicates(['申请号','专利类型'],keep='last')
        tech_net_category =  doc_lda_w2v_ct.loc[doc_lda_w2v_ct.index.isin(filtered_value_year['公开（公告）号'])]
    
        if tech_net_category.empty: 
            continue
            
        tech_net_category = tech_net_category.groupby(tech_net_category.index).mean()    #合并该公司重复索引的专利vec
        temp_results = {}
        for n in lookup_year:
            base_vec=base_tech_net.get(n, pd.Series())
            if not base_vec.empty:
                cosine_sim_values = cosine_similarity(tech_net_category.values, base_vec.values)
                tech_net_co = pd.Series(cosine_sim_values.mean(axis=1), index=tech_net_category.index)      
            
                if tech_net_co.isnull().all():
                    continue
                
                for th in thresholds:
                    temp_results.update({
                        f'tech_net_Explorative{n}_{th}': (tech_net_co < th).mean(),
                        f'tech_net_Exploitative{n}_{th}': (tech_net_co >= th).mean(),
                    })
                    
        if temp_results:
            invest_co_ct.loc[index, temp_results.keys()] = pd.Series(temp_results) 
  

In [30]:
def chunked_mean_safe_cosine_similarity(source, target, chunk_size=1000, memory_limit=10):
    memory_usage_gb = (source.shape[0] * target.shape[0] * 8) / (1024**3) #estimated
    #memory_gb = (source.nbytes + target.nbytes) / (1024**3)
    if memory_usage_gb > memory_limit * 2:
        source = source.astype(np.float16)
        target = target.astype(np.float16)
    elif memory_usage_gb > memory_limit:
        source = source.astype(np.float32)
        target = target.astype(np.float32)
        
    results = []
    start_idx = 0
    while chunk_size > 1: 
        try:
            for i in range(start_idx, source.shape[0], chunk_size):
                chunk = source[i:i + chunk_size]
                chunk_result = cosine_similarity(chunk, target)
                results.append(chunk_result.mean(axis=1))
                memory_usage_gb = chunk_result.nbytes / (1024**3)
                if memory_usage_gb * 2.2 > memory_limit: # current_memory_usage_gb*2 预留空间
                    print('.', end='', flush=True)
                    del chunk_result
                    gc.collect()
            break
        except MemoryError:
            print(f"\nMemoryError: Reducing chunk_size from {chunk_size} to {chunk_size // 2},memory usage {memory_usage_gb:.2f} GB")
            chunk_size = max(1, chunk_size // 2)
            start_idx = i  # 从未完成部分重新开始处理
        
    return np.concatenate(results)

In [11]:
import gc

In [12]:
def chunked_safe_cosine_similarity_faiss(source, target, index, public_id_to_faiss_idx, chunk_size=1000, memory_limit_gb=15):
    #分块计算两组公开号的余弦相似度，使用 FAISS 批量处理以避免内存问题。
    if len(target)>chunk_size:
        memory_usage_gb = (len(source) * chunk_size * 4) / (1024**3) #estimated
        if memory_usage_gb > memory_limit_gb:
            chunk_size = max(2,int(chunk_size * memory_limit_gb/memory_usage_gb))
    else:
        memory_usage_gb = (len(source) * len(target) * 4) / (1024**3)
        if memory_usage_gb > memory_limit_gb:
            chunk_size = max(2,int(len(target) * memory_limit_gb/memory_usage_gb))
        
    results = []
    start_idx = 0
    while chunk_size > 1: 
        try:
            for i in range(start_idx, len(source), chunk_size):
                chunk_source = source[i:i + chunk_size]
                chunk_result = batch_faiss_cos_sim(chunk_source, target, index, public_id_to_faiss_idx) #cosine_similarity(chunk, target)
                
                results.append(chunk_result.mean(axis=1))
                memory_usage_gb = chunk_result.nbytes / (1024**3)
                if memory_usage_gb * 2.2 > memory_limit_gb: # current_memory_usage_gb*2 预留空间
                    print('.', end='', flush=True)
                    del chunk_result
                    gc.collect()
            break
        except MemoryError:
            print(f"\nMemoryError: Reducing chunk_size from {chunk_size} to {chunk_size // 2},memory usage {memory_usage_gb:.2f} GB")
            chunk_size = max(1, chunk_size // 2)
            start_idx = i  # 从未完成部分重新开始处理
        
    return np.concatenate(results)

In [35]:
thresholds_exploitive=np.arange(0.7,0.8,0.02)
thresholds_explorative=np.arange(0.2,0.31,0.02)
thresholds_exploitive,thresholds_explorative

(array([0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 ]),
 array([0.2 , 0.22, 0.24, 0.26, 0.28, 0.3 ]))

In [48]:
#invest_co_xj.columns
columns_to_drop = [col for col in invest_co_xj.columns if col.startswith('tech_net_Exp')]
columns_to_drop

['tech_net_Explorative1_0.7',
 'tech_net_Exploitative1_0.7',
 'tech_net_Explorative1_0.75',
 'tech_net_Exploitative1_0.75',
 'tech_net_Explorative1_0.8',
 'tech_net_Exploitative1_0.8',
 'tech_net_Explorative1_0.85',
 'tech_net_Exploitative1_0.85',
 'tech_net_Explorative2_0.7',
 'tech_net_Exploitative2_0.7',
 'tech_net_Explorative2_0.75',
 'tech_net_Exploitative2_0.75',
 'tech_net_Explorative2_0.8',
 'tech_net_Exploitative2_0.8',
 'tech_net_Explorative2_0.85',
 'tech_net_Exploitative2_0.85',
 'tech_net_Explorative3_0.7',
 'tech_net_Exploitative3_0.7',
 'tech_net_Explorative3_0.75',
 'tech_net_Exploitative3_0.75',
 'tech_net_Explorative3_0.8',
 'tech_net_Exploitative3_0.8',
 'tech_net_Explorative3_0.85',
 'tech_net_Exploitative3_0.85',
 'tech_net_Explorative4_0.7',
 'tech_net_Exploitative4_0.7',
 'tech_net_Explorative4_0.75',
 'tech_net_Exploitative4_0.75',
 'tech_net_Explorative4_0.8',
 'tech_net_Exploitative4_0.8',
 'tech_net_Explorative4_0.85',
 'tech_net_Exploitative4_0.85',
 'tech_n

In [49]:
invest_co_xj = invest_co_xj.drop(columns=columns_to_drop)

In [81]:
del doc_lda_w2v_xj
del doc_vec_xj
gc.collect()

2810598

In [19]:
columns_to_drop = [col for col in invest_co_xj.columns if col.endswith('std')]
columns_to_drop

['tech_net_Explorative1_std',
 'tech_net_Explorative2_std',
 'tech_net_Explorative3_std',
 'tech_net_Explorative4_std',
 'tech_net_Explorative5_std']

In [20]:
invest_co_xj = invest_co_xj.drop(columns=columns_to_drop)

In [ ]:
%%time
lookup_year=[1,2,3,4,5]
thresholds_exploitive=np.arange(0.7,0.81,0.01)
thresholds_explorative=np.arange(0.65,0.76,0.01)

for year in sorted(invest_co_xj.交易时间.dt.year.unique()):
    # if year<=2018:
    #     continue
    # max_date=pd.to_datetime(year, format='%Y')+ pd.offsets.YearEnd()
    filter_invest = invest_co_xj[invest_co_xj.交易时间.dt.year==year]
    filter_patent = patent_co[(patent_co.先进制造==1)&(patent_co.申请年 <= year)]# <= data
    
    if filter_patent.empty:
        continue
 
    # mask_base_tech_net ={n: doc_lda_w2v_xj.index.isin(filter_patent.loc[filter_patent['申请年'].between(year - n, year - 1),'公开（公告）号']) 
    #                 for n in lookup_year}
    base_tech_net_ids ={n: filter_patent.loc[filter_patent['申请年'].between(year - n, year - 1),'公开（公告）号'].tolist() for n in lookup_year}
   
    for index,row in filter_invest.iterrows():
        company_name = row['企业全称']
        date = row['交易时间']

        filtered_value_year = filter_patent[(filter_patent.Co==company_name)&(filter_patent['申请年'] ==  year)].sort_values(['申请日','公开（公告）日']).drop_duplicates(['申请号','专利类型'],keep='last')
        # tech_net_category =  doc_lda_w2v_xj.loc[doc_lda_w2v_xj.index.isin(filtered_value_year['公开（公告）号'])]
        # if tech_net_category.empty: 
        #     continue
            
        # tech_net_category = tech_net_category.groupby(tech_net_category.index).mean()    #合并该公司重复索引的专利vec
        tech_net_category_ids =filtered_value_year['公开（公告）号'].tolist()
        if not tech_net_category_ids:
            continue
            
        temp_results = {}
        for n in lookup_year:
            # base_tech_net= doc_lda_w2v_xj.loc[mask_base_tech_net[n]]
            # if base_tech_net.empty:
            #     continue
            # base_tech_net = base_tech_net.groupby(base_tech_net.index).mean()    
            # #cosine_sim_values = cosine_similarity(tech_net_category.values, base_tech_net.values)#cosine_sim_values.mean(axis=1)
            # tech_cosine_sim_mean=chunked_mean_safe_cosine_similarity(tech_net_category, base_tech_net, chunk_size=1000, memory_limit=10)
            if not base_tech_net_ids[n]:
                continue
                
            tech_cosine_sim_mean=chunked_safe_cosine_similarity_faiss(source=tech_net_category_ids, target=base_tech_net_ids[n], 
                                                                      index=index_xj, public_id_to_faiss_idx=public_id_to_faiss_idx_xj, chunk_size=1000, memory_limit_gb=16)
            # tech_net_co = pd.Series(tech_cosine_sim_mean, index=tech_net_category.index)      
            # if tech_net_co.isnull().all():
            #     continue
            # for th in thresholds:
            #     #(cosine_sim_values < th).mean(axis=1)  每个目标专利在参考专利小于 th 的比例 .mean(),目标专利公司探索性比例的平均
            #     temp_results.update({
            #         f'tech_net_Explorative{n}_{th}': (tech_net_co < th).mean(),#目标专利公司整体平均相似度小于 th 的比例
            #         f'tech_net_Exploitative{n}_{th}': (tech_net_co >= th).mean(),#互补和为1
            #     })
            if tech_cosine_sim_mean.size == 0:
                continue
                
            #(cosine_sim_values < th).mean(axis=1)  每个目标专利在参考专利小于 th 的比例 .mean(),目标专利公司探索性比例的平均
            temp_results.update({f'tech_net_Exploitive{n}_{round(th, 2)}': (tech_cosine_sim_mean < th).mean() for th in thresholds_exploitive})#0.7-0.8
            temp_results.update({f'tech_net_Explorative{n}_{round(th, 2)}': (tech_cosine_sim_mean > th).mean() for th in thresholds_explorative})#0.2-0.3
            temp_results[f'tech_net_std{n}'] = tech_cosine_sim_mean.std()
        
        if temp_results:
            invest_co_xj.loc[index, temp_results.keys()] = pd.Series(temp_results) 
  
        
    print(year,filter_patent['Co'].nunique(),filter_invest['企业全称'].nunique())
# CPU times: total: 14h 57min 23s
# Wall time: 4h 26min 49s

1992 4 4
1993 5 9
1994 5 4
1995 10 6
1996 17 13
1997 22 14
1998 28 10
1999 42 2
2000 60 27
2001 109 27
2002 163 34
2003 229 36
2004 312 37
2005 416 31
2006 561 45
2007 751 91
2008 1039 114
2009 1332 125
2010 1653 309
2011 2002 362
2012 2376 323
2013 2740 310
2014 3145 837
2015 3652 1536
2016 4215 2144


In [22]:
invest_co_xj.columns

Index(['行业', '子行业', '企业全称', '交易时间', '交易轮次', 'all', 'explorative',
       'exploitative', 'jianjin_patent_num', '首次上市日期',
       ...
       'tech_net_Explorative5_0.71', 'tech_net_Explorative5_0.72',
       'tech_net_Explorative5_0.73', 'tech_net_Explorative5_0.74',
       'tech_net_Explorative5_0.75', 'tech_net_std1', 'tech_net_std2',
       'tech_net_std3', 'tech_net_std4', 'tech_net_std5'],
      dtype='object', length=217)

In [72]:
def rename_exp(col):
    parts = col.split('_')
    last_part = parts[-1]
    try:
        value = float(last_part)
        rounded_value = round(value, 2)
        if 0.7 <= value <= 0.8:
            parts[2]=parts[2].replace('Explorative',"Exploitive")
        if 0.2 <= value <= 0.3:
            parts[2]=parts[2].replace("Exploitive",'Explorative')
        parts[-1] = str(rounded_value)
    except ValueError:
        pass
    return '_'.join(parts)
    
rename_columns = {col: rename_exp(col)  for col in invest_co_xj.filter(like='tech_net_Exp').columns}
rename_columns

{'tech_net_Explorative1_0.7': 'tech_net_Exploitive1_0.7',
 'tech_net_Explorative1_0.72': 'tech_net_Exploitive1_0.72',
 'tech_net_Explorative1_0.74': 'tech_net_Exploitive1_0.74',
 'tech_net_Explorative1_0.76': 'tech_net_Exploitive1_0.76',
 'tech_net_Explorative1_0.78': 'tech_net_Exploitive1_0.78',
 'tech_net_Explorative1_0.8': 'tech_net_Exploitive1_0.8',
 'tech_net_Exploitive1_0.2': 'tech_net_Explorative1_0.2',
 'tech_net_Exploitive1_0.22': 'tech_net_Explorative1_0.22',
 'tech_net_Exploitive1_0.24': 'tech_net_Explorative1_0.24',
 'tech_net_Exploitive1_0.26': 'tech_net_Explorative1_0.26',
 'tech_net_Exploitive1_0.28': 'tech_net_Explorative1_0.28',
 'tech_net_Exploitive1_0.3': 'tech_net_Explorative1_0.3',
 'tech_net_Explorative2_0.7': 'tech_net_Exploitive2_0.7',
 'tech_net_Explorative2_0.72': 'tech_net_Exploitive2_0.72',
 'tech_net_Explorative2_0.74': 'tech_net_Exploitive2_0.74',
 'tech_net_Explorative2_0.76': 'tech_net_Exploitive2_0.76',
 'tech_net_Explorative2_0.78': 'tech_net_Exploitiv

In [73]:
invest_co_xj.rename(columns=rename_columns, inplace=True)

In [25]:
invest_co_xj.filter(like='tech_net_Explorative').describe()

,tech_net_Explorative1_0.2,tech_net_Explorative1_0.22,tech_net_Explorative1_0.24,tech_net_Explorative1_0.26,tech_net_Explorative1_0.28,tech_net_Explorative1_0.3,tech_net_Explorative2_0.2,tech_net_Explorative2_0.22,tech_net_Explorative2_0.24,tech_net_Explorative2_0.26,...,tech_net_Explorative5_0.66,tech_net_Explorative5_0.67,tech_net_Explorative5_0.68,tech_net_Explorative5_0.69,tech_net_Explorative5_0.7,tech_net_Explorative5_0.71,tech_net_Explorative5_0.72,tech_net_Explorative5_0.73,tech_net_Explorative5_0.74,tech_net_Explorative5_0.75
count,13171.0,13171.0,13171.0,13171.0,13171.0,13171.0,13171.0,13171.0,13171.0,13171.0,...,13171.000000,13171.000000,13171.000000,13171.000000,13171.000000,13171.000000,13171.000000,13171.000000,13171.000000,13171.000000
mean,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.900020,0.867731,0.831114,0.788947,0.742774,0.690416,0.629967,0.560898,0.481396,0.394409
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.201902,0.229404,0.255881,0.280417,0.300456,0.317460,0.328111,0.331177,0.326703,0.310139
min,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.892857,0.830986,0.750000,0.666667,0.583333,0.500000,0.400000,0.321429,0.204732,0.117647
50%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.000000,1.000000,0.961538,0.909091,0.851852,0.783784,0.700000,0.600000,0.500000,0.380282
75%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.925926,0.833333,0.738613,0.608696
max,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [23]:
invest_co_xj.filter(like='tech_net_Exploitive').describe()

,tech_net_Exploitive1_0.7,tech_net_Exploitive1_0.72,tech_net_Exploitive1_0.74,tech_net_Exploitive1_0.76,tech_net_Exploitive1_0.78,tech_net_Exploitive1_0.8,tech_net_Exploitive2_0.7,tech_net_Exploitive2_0.72,tech_net_Exploitive2_0.74,tech_net_Exploitive2_0.76,...,tech_net_Exploitive4_0.75,tech_net_Exploitive4_0.77,tech_net_Exploitive4_0.79,tech_net_Exploitive4_0.81,tech_net_Exploitive5_0.71,tech_net_Exploitive5_0.73,tech_net_Exploitive5_0.75,tech_net_Exploitive5_0.77,tech_net_Exploitive5_0.79,tech_net_Exploitive5_0.81
count,13171.000000,13171.000000,13171.000000,13171.000000,13171.000000,13171.000000,13171.000000,13171.000000,13171.000000,13171.000000,...,13171.000000,13171.000000,13171.000000,13171.000000,13171.000000,13171.000000,13171.000000,13171.000000,13171.000000,13171.000000
mean,0.251692,0.359184,0.500895,0.679519,0.862174,0.978165,0.252926,0.361276,0.504542,0.683774,...,0.600022,0.787444,0.939219,0.996743,0.309584,0.439102,0.605586,0.792607,0.941990,0.996938
std,0.299354,0.328624,0.328802,0.288967,0.196436,0.077649,0.299804,0.328555,0.328239,0.287823,...,0.312176,0.243257,0.128510,0.034534,0.317460,0.331177,0.310138,0.241087,0.126170,0.033788
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.057143,0.250000,0.500000,0.791667,1.000000,0.000000,0.058824,0.250000,0.500000,...,0.379049,0.666667,0.923077,1.000000,0.000000,0.166667,0.391304,0.666667,0.928571,1.000000
50%,0.142857,0.285714,0.500000,0.714286,0.937500,1.000000,0.142857,0.285714,0.500000,0.725806,...,0.607843,0.850000,1.000000,1.000000,0.216216,0.400000,0.619048,0.857143,1.000000,1.000000
75%,0.400000,0.583333,0.765862,1.000000,1.000000,1.000000,0.400000,0.586492,0.769231,1.000000,...,0.878788,1.000000,1.000000,1.000000,0.500000,0.678571,0.882353,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [24]:
invest_co_xj.to_excel('data/invest_patent_先进制造_202410_5.xlsx')
invest_co_xj.columns

Index(['行业', '子行业', '企业全称', '交易时间', '交易轮次', 'all', 'explorative',
       'exploitative', 'jianjin_patent_num', '首次上市日期',
       ...
       'tech_net_Explorative5_0.71', 'tech_net_Explorative5_0.72',
       'tech_net_Explorative5_0.73', 'tech_net_Explorative5_0.74',
       'tech_net_Explorative5_0.75', 'tech_net_std1', 'tech_net_std2',
       'tech_net_std3', 'tech_net_std4', 'tech_net_std5'],
      dtype='object', length=217)

In [89]:
del vectors
del index_xj

In [26]:
doc_lda_w2v_ct=pd.read_parquet(f'data\\documents_vec_invest_mfg_ct.parquet')
doc_lda_w2v_ct.shape

(1043952, 300)

In [90]:
%%time
doc_vec_ct=doc_lda_w2v_ct.groupby(doc_lda_w2v_ct.index).mean()#合并复索引的专利vec
public_id_to_faiss_idx_ct = {pid: idx for idx, pid in enumerate(doc_vec_ct.index)}
vectors = doc_vec_ct.values.astype(np.float32) #np.stack
faiss.normalize_L2(vectors)#归一化向量，使点积 = 余弦相似度
index_ct = faiss.IndexFlatIP(vectors.shape[1])# faiss.IndexFlatL2
index_ct.add(vectors)
vectors.shape

CPU times: total: 7.58 s
Wall time: 7.05 s


(1023740, 300)

In [91]:
with open('data\\doc_vec_invest_mfg_ct_public_id_to_faiss.pkl', 'wb') as f:
    pickle.dump(public_id_to_faiss_idx_ct, f)
faiss.write_index(index_ct, 'data\\doc_vec_invest_mfg_ct.faiss')#faiss.read_index

In [92]:
columns_to_drop = [col for col in invest_co_ct.columns if col.startswith('tech_net_Exp')]
invest_co_ct = invest_co_ct.drop(columns=columns_to_drop)

In [93]:
del doc_lda_w2v_ct
del doc_vec_ct
gc.collect()

3623375

In [29]:
gc.collect()

6070745

In [28]:
index_ct=faiss.read_index('data\\doc_vec_invest_mfg_ct.faiss')
with open('data\\doc_vec_invest_mfg_ct_public_id_to_faiss.pkl', 'rb') as f:
    public_id_to_faiss_idx_ct=pickle.load(f)

In [30]:
%%time
lookup_year=[1,2,3,4,5]
thresholds_exploitive=np.arange(0.7,0.81,0.01)
thresholds_explorative=np.arange(0.65,0.76,0.01)

for year in sorted(invest_co_ct.交易时间.dt.year.unique()):
    filter_invest = invest_co_ct[invest_co_ct.交易时间.dt.year==year]
    filter_patent = patent_co[(patent_co.传统制造==1)&(patent_co.申请年 <= year)]# <= data
    
    if filter_patent.empty:
        continue

    base_tech_net_ids ={n: filter_patent.loc[filter_patent['申请年'].between(year - n, year - 1),'公开（公告）号'].tolist() for n in lookup_year}
   
    for index,row in filter_invest.iterrows():
        company_name = row['企业全称']

        filtered_value_year = filter_patent[(filter_patent.Co==company_name)&(filter_patent['申请年'] ==  year)].sort_values(['申请日','公开（公告）日']).drop_duplicates(['申请号','专利类型'],keep='last')
        tech_net_category_ids =filtered_value_year['公开（公告）号'].tolist()
        if not tech_net_category_ids:
            continue
            
        temp_results = {}
        for n in lookup_year:
            if not base_tech_net_ids[n]:
                continue
                
            tech_cosine_sim_mean = chunked_safe_cosine_similarity_faiss(source=tech_net_category_ids, target=base_tech_net_ids[n], 
                                                                      index=index_ct, public_id_to_faiss_idx=public_id_to_faiss_idx_ct, chunk_size=1000, memory_limit_gb=16)
            if tech_cosine_sim_mean.size == 0:
                continue
                
            temp_results.update({f'tech_net_Exploitive{n}_{round(th, 2)}': (tech_cosine_sim_mean < th).mean() for th in thresholds_exploitive})#0.7-0.8
            temp_results.update({f'tech_net_Explorative{n}_{round(th, 2)}': (tech_cosine_sim_mean > th).mean() for th in thresholds_explorative})
            temp_results[f'tech_net_std{n}'] = tech_cosine_sim_mean.std()
            
        if temp_results:
            invest_co_ct.loc[index, temp_results.keys()] = pd.Series(temp_results) 
  
        
    print(year,filter_patent['Co'].nunique(),filter_invest['企业全称'].nunique())
# CPU times: total: 4h 49min 39s
# Wall time: 1h 25min 33s

1990 4 1
1992 5 2
1993 10 9
1994 13 7
1995 17 2
1996 26 25
1997 39 23
1998 51 16
1999 70 10
2000 93 75
2001 150 56
2002 208 41
2003 288 64
2004 402 72
2005 555 79
2006 820 113
2007 1103 226
2008 1527 266
2009 1972 249
2010 2431 599
2011 2899 687
2012 3359 508
2013 3809 408
2014 4229 813
2015 4645 1731
2016 4999 2249
2017 5278 1679
2018 5450 1276
2019 5581 1133
2020 5702 1139
2021 7600 1082
2022 7840 1395
2023 8013 1342
2024 8017 271
CPU times: total: 5h 21min 33s
Wall time: 1h 45min 52s


In [31]:
invest_co_ct.to_excel('data/invest_patent_传统制造_202410_5.xlsx')

In [32]:
invest_co_ct.shape

(19872, 197)

In [97]:
invest_co_ct.filter(like='tech_net_Explorative').describe()

,tech_net_Explorative1_0.5,tech_net_Explorative1_0.52,tech_net_Explorative1_0.54,tech_net_Explorative1_0.56,tech_net_Explorative1_0.58,tech_net_Explorative1_0.6,tech_net_Explorative2_0.5,tech_net_Explorative2_0.52,tech_net_Explorative2_0.54,tech_net_Explorative2_0.56,...,tech_net_Explorative4_0.62,tech_net_Explorative4_0.64,tech_net_Explorative4_0.66,tech_net_Explorative4_0.68,tech_net_Explorative4_0.7,tech_net_Explorative5_0.62,tech_net_Explorative5_0.64,tech_net_Explorative5_0.66,tech_net_Explorative5_0.68,tech_net_Explorative5_0.7
count,9006.000000,9006.000000,9006.000000,9006.000000,9006.000000,9006.000000,9006.000000,9006.000000,9006.000000,9006.000000,...,9006.000000,9006.000000,9006.000000,9006.000000,9006.000000,9006.000000,9006.000000,9006.000000,9006.000000,9006.000000
mean,0.999911,0.999897,0.999845,0.998474,0.993315,0.986280,0.999911,0.999902,0.999847,0.998596,...,0.976609,0.957331,0.927807,0.884174,0.833549,0.977472,0.958083,0.928484,0.884950,0.834202
std,0.004750,0.004831,0.005611,0.022136,0.051530,0.077807,0.004750,0.004783,0.005545,0.021414,...,0.102344,0.140297,0.183689,0.229945,0.270644,0.100196,0.138944,0.183040,0.229275,0.269922
min,0.684211,0.684211,0.684211,0.000000,0.000000,0.000000,0.684211,0.684211,0.684211,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,0.983607,0.875000,0.750000,1.000000,1.000000,0.986967,0.875000,0.750000
50%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [106]:
a=invest_co_ct.filter(like='tech_net_Explorative')
((a==0)|(a==1)).sum().sort_index(axis=0)

tech_net_Explorative1_0.5     9002
tech_net_Explorative1_0.52    8996
tech_net_Explorative1_0.54    8992
tech_net_Explorative1_0.56    8900
tech_net_Explorative1_0.58    8642
tech_net_Explorative1_0.6     8383
tech_net_Explorative1_0.62    8004
tech_net_Explorative1_0.64    7456
tech_net_Explorative1_0.66    6780
tech_net_Explorative1_0.68    6059
tech_net_Explorative1_0.7     5368
tech_net_Explorative2_0.5     9002
tech_net_Explorative2_0.52    8997
tech_net_Explorative2_0.54    8991
tech_net_Explorative2_0.56    8910
tech_net_Explorative2_0.58    8646
tech_net_Explorative2_0.6     8391
tech_net_Explorative2_0.62    8018
tech_net_Explorative2_0.64    7477
tech_net_Explorative2_0.66    6808
tech_net_Explorative2_0.68    6070
tech_net_Explorative2_0.7     5368
tech_net_Explorative3_0.5     9003
tech_net_Explorative3_0.52    8998
tech_net_Explorative3_0.54    8993
tech_net_Explorative3_0.56    8913
tech_net_Explorative3_0.58    8650
tech_net_Explorative3_0.6     8392
tech_net_Explorative

In [107]:
b=invest_co_ct.filter(like='tech_net_Exploitive')
((b==0)|(b==1)).sum().sort_index(axis=0)

tech_net_Exploitive1_0.7     5368
tech_net_Exploitive1_0.72    4580
tech_net_Exploitive1_0.74    3455
tech_net_Exploitive1_0.76    2765
tech_net_Exploitive1_0.78    3127
tech_net_Exploitive1_0.8     5813
tech_net_Exploitive2_0.7     5368
tech_net_Exploitive2_0.72    4570
tech_net_Exploitive2_0.74    3445
tech_net_Exploitive2_0.76    2746
tech_net_Exploitive2_0.78    3149
tech_net_Exploitive2_0.8     5844
tech_net_Exploitive3_0.7     5369
tech_net_Exploitive3_0.72    4557
tech_net_Exploitive3_0.74    3425
tech_net_Exploitive3_0.76    2735
tech_net_Exploitive3_0.78    3133
tech_net_Exploitive3_0.8     5881
tech_net_Exploitive4_0.7     5368
tech_net_Exploitive4_0.72    4543
tech_net_Exploitive4_0.74    3414
tech_net_Exploitive4_0.76    2729
tech_net_Exploitive4_0.78    3145
tech_net_Exploitive4_0.8     5949
tech_net_Exploitive5_0.7     5367
tech_net_Exploitive5_0.72    4524
tech_net_Exploitive5_0.74    3390
tech_net_Exploitive5_0.76    2732
tech_net_Exploitive5_0.78    3174
tech_net_Explo

In [102]:
ia=((a>0)&(a<1)).sum().sort_index(axis=0)
ia

tech_net_Explorative1_0.5        4
tech_net_Explorative1_0.52      10
tech_net_Explorative1_0.54      14
tech_net_Explorative1_0.56     106
tech_net_Explorative1_0.58     364
tech_net_Explorative1_0.6      623
tech_net_Explorative1_0.62    1002
tech_net_Explorative1_0.64    1550
tech_net_Explorative1_0.66    2226
tech_net_Explorative1_0.68    2947
tech_net_Explorative1_0.7     3638
tech_net_Explorative2_0.5        4
tech_net_Explorative2_0.52       9
tech_net_Explorative2_0.54      15
tech_net_Explorative2_0.56      96
tech_net_Explorative2_0.58     360
tech_net_Explorative2_0.6      615
tech_net_Explorative2_0.62     988
tech_net_Explorative2_0.64    1529
tech_net_Explorative2_0.66    2198
tech_net_Explorative2_0.68    2936
tech_net_Explorative2_0.7     3638
tech_net_Explorative3_0.5        3
tech_net_Explorative3_0.52       8
tech_net_Explorative3_0.54      13
tech_net_Explorative3_0.56      93
tech_net_Explorative3_0.58     356
tech_net_Explorative3_0.6      614
tech_net_Explorative

In [108]:
ib=((b>0)&(b<1)).sum().sort_index(axis=0)
ib

tech_net_Exploitive1_0.7     3638
tech_net_Exploitive1_0.72    4426
tech_net_Exploitive1_0.74    5551
tech_net_Exploitive1_0.76    6241
tech_net_Exploitive1_0.78    5879
tech_net_Exploitive1_0.8     3193
tech_net_Exploitive2_0.7     3638
tech_net_Exploitive2_0.72    4436
tech_net_Exploitive2_0.74    5561
tech_net_Exploitive2_0.76    6260
tech_net_Exploitive2_0.78    5857
tech_net_Exploitive2_0.8     3162
tech_net_Exploitive3_0.7     3637
tech_net_Exploitive3_0.72    4449
tech_net_Exploitive3_0.74    5581
tech_net_Exploitive3_0.76    6271
tech_net_Exploitive3_0.78    5873
tech_net_Exploitive3_0.8     3125
tech_net_Exploitive4_0.7     3638
tech_net_Exploitive4_0.72    4463
tech_net_Exploitive4_0.74    5592
tech_net_Exploitive4_0.76    6277
tech_net_Exploitive4_0.78    5861
tech_net_Exploitive4_0.8     3057
tech_net_Exploitive5_0.7     3639
tech_net_Exploitive5_0.72    4482
tech_net_Exploitive5_0.74    5616
tech_net_Exploitive5_0.76    6274
tech_net_Exploitive5_0.78    5832
tech_net_Explo

In [101]:
sa=a[(a>0)&(a<1)].std().sort_index(axis=0)
sa

tech_net_Explorative1_0.5     0.121166
tech_net_Explorative1_0.52    0.117453
tech_net_Explorative1_0.54    0.105318
tech_net_Explorative1_0.56    0.134594
tech_net_Explorative1_0.58    0.167977
tech_net_Explorative1_0.6     0.180765
tech_net_Explorative1_0.62    0.184178
tech_net_Explorative1_0.64    0.193068
tech_net_Explorative1_0.66    0.196828
tech_net_Explorative1_0.68    0.207352
tech_net_Explorative1_0.7     0.214084
tech_net_Explorative2_0.5     0.121166
tech_net_Explorative2_0.52    0.121829
tech_net_Explorative2_0.54    0.103786
tech_net_Explorative2_0.56    0.135133
tech_net_Explorative2_0.58    0.168895
tech_net_Explorative2_0.6     0.182623
tech_net_Explorative2_0.62    0.187169
tech_net_Explorative2_0.64    0.191109
tech_net_Explorative2_0.66    0.197830
tech_net_Explorative2_0.68    0.205484
tech_net_Explorative2_0.7     0.215397
tech_net_Explorative3_0.5     0.058074
tech_net_Explorative3_0.52    0.127885
tech_net_Explorative3_0.54    0.107844
tech_net_Explorative3_0.5